In [2]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import numpy as np

In [4]:
units = pd.read_csv("../data/510049986/raw_data.csv", encoding="windows-1252", index_col=0)
hashes = pd.read_csv("../raws/510053208/ZZ_Win/pc/localisation/us/localisation/unites_fixed.csv", 
                     encoding='windows-1252', 
                     index_col=0)

C:\Users\Alex\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13,28,43,50,51,60,62,66,68,69,73,77,79,80,82,83,84,85,86,87,90,91,92,93,97,98,101,107,109,110,111,112,113,114,118,120,121,122,124,125,127,128,132,133,134,135,136,137,139,140,144,148,150,151,153,154,155,156,157,158,161,162,163,164,168,169,172,178,180,181,182,183,184,185,189,191,192,193,196,198,203,204,205,206,207,208,210,211,215,219,221,222,224,225,226,227,228,229,232,233,234,235,239,240,243,249,251,252,253,254,255,256,260,262,263,264,267,269,274,275,276,277,278,279,281,282,286,290,292,293,295,296,297,298,299,303,304,305,306,310,311,314,322,323,324,325,326,327,331,333,334,335,337,338,340,341,345,346,347,348,349,350,352,353,357,361,363,364,366,367,368,369,370,374,375,376,377,381,382,385,393,394,395,396,397,398,402,404,405,406,408,409,411,412,416,417,418,419,420,421,431,445,446,460,485,486,488,504,506,518,519,558,573,574,577,579,582,591,592,627,632,634,646,650,652,662,663,698,702,713,71

In [5]:
hashes.head()

,Hash,String
0,DC02000000000000,Anti-Radar
1,4203000000000000,C1A1 SLR
2,4303000000000000,C2A1
3,9A03000000000000,Type 64 LMG
4,4405000000000000,K3 LMG


In [13]:
decks = {
    '8BD43C9757360E00': 'Mechanized',
    '5C76718B57360E00': 'Armored',
    '5E767965E3000000': 'Motorized',
    'DAD77965E3000000': 'Support',
    '23B8605ED9380000': 'Marine',
    '0BB7685ED9380000': 'Airborne'
}


units['DeckTypes'] = units['UnitTypeTokens'].map(eval).map(lambda l: [decks[i] for i in l])

In [17]:
movers = {
    1: 'Foot',
    2: 'Wheeled',
    3: 'Wheeled',
    5: 'Tracked',
    6: 'Air',
    7: 'Wheeled',
    8: 'Tracked',
    9: 'Water'
}
amphib = {
    1: False,
    2: False,
    3: False,
    5: False,
    6: np.nan,
    7: True,
    8: True,
    9: np.nan
}

units['MovementType'] = units['UnitMovingType'].map(movers)
units['Amphibious'] = units['UnitMovingType'].map(amphib)

In [20]:
units['MovementType'].value_counts(dropna=False)

Tracked    670
Air        448
Wheeled    372
Foot       296
Water       32
Name: MovementType, dtype: int64

In [23]:
training = {
    '8F37594F19619C07' : 'Elite',
    '5593495D19619C07' : 'Shock',
    'D6173D5C19619C07' : 'Regular',
    'DE644D5719619C07' : 'Militia',
    'null': np.nan
}

units['TrainingLevel'] = units['Training'].map(training)

In [25]:
ciws = {
    '4F233E0000000000': 'Exceptional',
    '4E96452000000000': 'Very Good',
    '4E96450000000000': 'Good',
    'D672711906000000': 'Medium',
    'CEC2000000000000': 'Bad',
    'null': np.nan
}

units['CIWSLevel'] = units['CIWS'].map(ciws)

In [26]:
sailing = {
    'CBD32D65B4780000': 'Deep Sea',
    'CBD33165B4780000': 'Coastal',
    'CBD33565B4780000': 'Riverine',
    'null': np.nan
}
units['SailingType'] = units['Sailing'].map(sailing)

In [171]:
def weaponify(unit):
    srs = pd.Series()
    prev_wep_name = ""
    for i in range(1, 12):
        wep_name = unit['Weapon{0}Name'.format(i)]
        if pd.isnull(wep_name):
            return srs
        elif wep_name == prev_wep_name:
            # Two cases: HE-AP split, and smoke rounds.
            if (unit['Weapon{0}PhysicalDamages'.format(i)] == "null" or pd.isnan(unit['Weapon{0}PhysicalDamages'.format(i)]))\
                and (unit['Weapon{0}Arme'.format(i)].isin(["null", 3]) or pd.isnan(unit['Weapon{0}Arme'.format(i)])):  # smoke
                srs['CanSmoke'.format(i)] = True
            elif (unit['Weapon{0}PhysicalDamages'.format(i)] == "null" 
                  or pd.isnan(unit['Weapon{0}PhysicalDamages'.format(i)])):  # HE component
                srs['Arme'] = unit['Weapon{0}Arme'.format(i)]
            elif (unit['Weapon{0}Arme'] == "null" or pd.isnan(unit['Weapon{0}Arme'])):  # AP component
                srs['PhysicalDamages'] = unit['Weapon{0}PhysicalDamages'.format(i)]            
            else:
                raise ValueError("You failed!")
        else:
            pass

In [174]:
weaponify(units.iloc[0])

Series([], dtype: float64)